<b>Google Colab</b> <a href="https://colab.research.google.com/github/kirillzyusko/stock-predictions/blob/master/lab.ipynb">link</a>

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
store = pd.read_csv('store.csv')

def one_hot(dataframe, column):
    # use pd.concat to join the new columns with your original dataframe
    df = pd.concat([dataframe, pd.get_dummies(dataframe[column], prefix=column)], axis=1)
    # now drop the original column (you don't need it anymore)
    df.drop([column], axis=1, inplace=True)

    return df

def inner_join(df1, df2, column):
    return pd.concat([df1.set_index(column),df2.set_index(column)], axis=1, join='inner').reset_index()

def where_equal(df, column, value):
    return df[df[column] == value]

def get_data():
    df = one_hot(train, 'DayOfWeek')

    df = inner_join(df, store, 'Store')
    df = one_hot(df, 'StoreType')
    df = one_hot(df, 'Assortment')
    df = one_hot(df, 'Store')

    df.drop([
        'Date',
        'Customers',
        'Promo',
        'PromoInterval',
        'Promo2',
        'Promo2SinceWeek',
        'Promo2SinceYear',
        'CompetitionDistance',
        'CompetitionOpenSinceMonth',
        'CompetitionOpenSinceYear'
    ], axis=1, inplace=True)

    # data = where_equal(df, 'Store', 1)
    # print(list(df))
    print(df.head())

    # df.to_csv('processed_train.csv', encoding='utf-8', index=False)

    return df

# get_data()

In [ ]:
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers
from keras.layers import Dropout
from sklearn.model_selection import train_test_split

import tensorflow as tf

from preprocessing import get_data

data = get_data()

y = data['Sales']
data.drop(['Sales'], axis=1, inplace=True)
X = data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

length, features = data.shape
stocks = 1115

with tf.device('/cpu:0'):
    model = Sequential()
    model.add(Dense(int(features * 1.5), input_dim=features, kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.001), activation='relu', name='hidden_1'))
    #model.add(Dropout(0.4, name='dropout_1'))
    model.add(Dense(features, kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(0.001), name='hidden_2'))
    #model.add(Dropout(0.4, name='dropout_2'))
    #model.add(Dense(int(features * 0.5), kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(0.001), name='hidden_3'))
    #model.add(Dropout(0.5, name='dropout_3'))
    model.add(Dense(1, kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.01), name='output_layer'))

    # Compile model
    model.compile(loss='mse', optimizer='adam', metrics=['acc'])

    print(model.summary())

    model.fit(X_train, y_train,
              epochs=50,
              verbose=1,
              validation_data=(X_test, y_test))

    score = model.evaluate(X_test, y_test, verbose=0)
    print(score)

Hello world 2


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('train.csv')
data = df[df['Store'] == 4]['Sales']

print(len(data))

plt.plot(data)
plt.show()